##### 실습N #1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.

In [ ]:
import FinanceDataReader as fdr
from io import StringIO
import os
import openai

# 엔비디아 한달치 주가 데이터 불러오기
df = fdr.DataReader('NVDA', '2023-07-21')

csv_buffer = StringIO()
df.to_csv(csv_buffer)

csv_data = csv_buffer.getvalue()
# print(csv_data)

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": """
      너는 주가 데이터 분석 전문가야.
      다음 데이터는 엔비디아 데이터야
      이 데이터를 보고 주가를 분석해줘 
      """
    },
    {
      "role": "user",
      "content": csv_data
    }
  ],
  temperature=0,
  max_tokens=1024,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)


엔비디아(NVIDIA)의 주가 데이터 분석을 진행하겠습니다. 주가 데이터는 2023년 7월 20일부터 2025년 7월 21일까지의 일일 주가 정보를 포함하고 있습니다. 이 데이터를 바탕으로 몇 가지 주요 포인트를 분석하겠습니다.

### 1. **주가 추세 분석**
- **상승세**: 2023년 7월부터 2024년 1월까지 주가는 전반적으로 상승세를 보였습니다. 특히 2023년 8월부터 2024년 1월까지의 기간 동안 주가는 급격히 상승하였으며, 2024년 1월에는 60달러를 초과하는 가격에 도달했습니다.
- **고점과 저점**: 2024년 2월에는 70달러를 초과하는 고점을 기록하였고, 이후 2024년 3월에는 90달러를 넘는 가격에 도달했습니다. 그러나 2024년 4월부터 2025년 1월까지는 주가가 하락세를 보이며 100달러 이하로 떨어졌습니다.
- **최근 동향**: 2025년 1월 이후 주가는 다시 상승세를 보이며 140달러를 초과하는 가격에 도달했습니다. 최근 2025년 7월까지의 데이터에서는 170달러 근처에서 거래되고 있습니다.

### 2. **거래량 분석**
- **거래량 변화**: 거래량은 주가의 변동성과 밀접한 관계가 있습니다. 주가가 급등하거나 급락할 때 거래량이 증가하는 경향이 있습니다. 예를 들어, 2024년 3월과 5월에는 거래량이 급증하며 주가가 상승했습니다.
- **거래량의 중요성**: 높은 거래량은 주가의 신뢰성을 높이며, 주가의 방향성을 확인하는 데 중요한 지표로 작용합니다.

### 3. **기술적 분석**
- **이동 평균선**: 단기 및 장기 이동 평균선을 활용하여 주가의 추세를 확인할 수 있습니다. 예를 들어, 50일 이동 평균선이 200일 이동 평균선을 상향 돌파하는 경우(골든 크로스)는 상승 신호로 해석될 수 있습니다.
- **RSI(상대 강도 지수)**: RSI를 통해 과매수 또는 과매도 상태를 판단할 수 있습니다. 일반적으로 RSI가 70 이상이면 과매수, 30 이하이면 과매도로 간주됩니다.

### 4. **

##### 실습N #2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오
##### 예 판교에 있는 괜찮은 이탈리안 식당을 몇개 추천해줄래 ?

In [15]:
import urllib.request
import urllib.parse
import os
import openai

client_id = "bN1aUL6XWa3mtO8SF9x3"
client_secret = "DiiJIjYDZW"

# ✅ 사용자 입력 예: "판교 이탈리안"
query = input("지역과 음식 종류를 입력하세요 (예: 판교 이탈리안): ").strip()
encoded_query = urllib.parse.quote(query)

# 🔗 네이버 로컬 검색 API 호출
url = f"https://openapi.naver.com/v1/search/local.json?query={encoded_query}&display=5"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

try:
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        raw_data = response.read().decode('utf-8')

        # OpenAI API 설정
        openai.api_key = os.getenv("OPENAI_API_KEY")

        # GPT에게 분석 요청
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": """
                    너는 지역이랑 음식 종류에 따라 음식을 추천해줘야해. 
                    사용자에게 입력한 지역에 있는 괜찮은 음식점을 추천해줘. JSON 데이터를 이해하고, 상호명·주소·특징 등을 바탕으로 요약하고 추천해.
                    """
                },
                {
                    "role": "user",
                    "content": raw_data
                }
            ],
            temperature=0,
            max_tokens=1024,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        print("추천 결과:")
        print(response.choices[0].message.content)

    else:
        print("rror Code:", rescode)

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} - {e.reason}")
    print(e.read().decode('utf-8'))


추천 결과:
부산에서 중식을 즐기고 싶으시다면 다음의 음식점을 추천드립니다:

1. **리산옥 화명본점**
   - **주소**: 부산광역시 북구 화명동 2272-3 2층 203호
   - **특징**: 중식당으로 다양한 중화요리를 제공합니다.

2. **구포짬뽕 부산구포본점**
   - **주소**: 부산광역시 북구 구포동 1212-9 1층 구포짬뽕
   - **특징**: 짬뽕 전문점으로, 깊고 진한 국물 맛이 일품입니다.
   - **링크**: [구포짬뽕 소개 영상](https://youtu.be/5IENQvYu-SU?si=OiLMnicVUzpC22qk)

3. **팔선생**
   - **주소**: 부산광역시 해운대구 우동 1483 센텀큐
   - **특징**: 해운대 지역의 중식당으로, 고급스러운 분위기에서 식사를 즐길 수 있습니다.

4. **홍유단**
   - **주소**: 부산광역시 사상구 괘법동 535-18 2층 홍유단
   - **특징**: 다양한 중화요리를 제공하며, 아늑한 분위기에서 식사할 수 있습니다.

5. **장성향**
   - **주소**: 부산광역시 동구 초량동 609-1
   - **특징**: 전통적인 중식 요리를 맛볼 수 있는 곳입니다.
   - **링크**: [장성향 웹사이트](http://jangseonghyang.com)

부산에서 맛있는 중식을 즐기시길 바랍니다!
